In [2]:
import config
import pymysql
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import pearsonr, spearmanr
from collections import Counter
from scipy.spatial.distance import euclidean
from dateutil.relativedelta import relativedelta

In [3]:
sid = "penguin_db"

In [7]:
con_saem = pymysql.connect(host = config.saem_db_m["host"],password = config.saem_db_m["password"],
                           user = config.saem_db_m["user"],db=sid,charset = "utf8")

In [86]:
con_saem.close()

Error: Already closed

In [29]:
pm = pd.read_sql("SELECT * FROM plant_master where use_yn='Y'and reg_prct_step>1",con=con_saem)
pm.columns=['pwp_id', 'usr_id', 'site_id', 'appl_date', 'saem_appl_date', 'pwp_div',
       'pwp_loct_grp_cd', 'addr_pnu_cd', 'addr_nm', 'pwp_nm', 'pwp_nick_nm',
       'mntr_id', 'mntr_lgn_id', 'mntr_lgn_pw', 'mntr_pwp_nm', 'mntr_alt_url',
       'tot_pv_cpct', 'tot_ess_cpct', 'tot_pcs_cpct', 'oprt_strt_date',
       'reg_prct_step', 'inverters', 'use_yn', 'del_yn', 'mdf_id', 'mdf_dtm',
       'reg_id', 'reg_dtm']

In [30]:
base_time = (datetime.datetime.today()-datetime.timedelta(days=31)).date()
base_time_str = base_time.strftime("%Y-%m-%d")
base_time2 = (datetime.datetime.today()-datetime.timedelta(days=1)).date()

In [31]:
base_time2

datetime.date(2020, 5, 6)

In [32]:
daily = pd.read_sql("SELECT A.*, B.pwp_loct_grp_cd FROM generation_log_daily as A, plant_master as B where A.stnd_date>='"+base_time_str+"' and A.pwp_id = B.id",con=con_saem)

In [34]:
daily["new_gen"] = np.log(daily.avg_pv_time+1).fillna(0)

In [35]:
daily_loc = daily[daily.stnd_date==base_time2]

In [69]:
# interquartile
up_loc = []
low_loc = []
for i in daily_loc.pwp_loct_grp_cd.unique():
    daily_loc2 = daily_loc[(daily_loc.pwp_loct_grp_cd==i)]
    daily_loc2["zscore"] = (daily_loc2.avg_pv_time-np.nanmean(daily_loc2.avg_pv_time))/np.nanstd(daily_loc2.avg_pv_time)
    daily_loc3 = daily_loc2[daily_loc2.avg_pv_time>0]
    
    upperb = 3
    lowerb = -3
    up_list = daily_loc2.pwp_id[daily_loc2.zscore>upperb].to_list()
    low_list = daily_loc2.pwp_id[daily_loc2.zscore<lowerb].to_list()
    up_loc += up_list
    low_loc += low_list

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [70]:
daily_loc

,id,pwp_id,stnd_date,pv_amt,avg_pv_time,rnk,tot_pv_cpct,rec_amt,smp_amt,rpt_dtm,reg_dtm,pwp_loct_grp_cd,new_gen,location,private
17766,1106619,8,2020-05-06,631.10,6.4039,NaN,98.55,33683.3216,46676.1560,2020-05-06 21:30:00,2020-05-06 22:00:01,45,2.002007,0.0,0.0
17767,1106620,9,2020-05-06,5840.08,5.8777,NaN,993.60,264944.2229,431932.3168,2020-05-06 20:30:00,2020-05-06 22:00:01,41,1.928284,0.0,0.0
17768,1106621,10,2020-05-06,443.00,6.8491,NaN,64.68,23643.9732,32764.2800,2020-05-06 21:30:00,2020-05-06 22:00:01,48,2.060399,0.0,0.0
17769,1106622,15,2020-05-06,3276.00,6.5843,NaN,497.55,151534.9181,242292.9600,2020-05-06 21:30:00,2020-05-06 22:00:01,41,2.026080,0.0,0.0
17770,1106623,18,2020-05-06,1361.00,5.4527,NaN,249.60,65375.8528,100659.5600,2020-05-06 21:30:00,2020-05-06 22:00:01,41,1.864499,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18356,1115425,1154,2020-05-06,675.00,6.8486,NaN,98.56,78374.2500,49923.0000,2020-05-07 14:36:17,2020-05-07 14:36:17,44,2.060335,0.0,0.0
18357,1115427,412,2020-05-06,673.00,6.7300,NaN,100.00,35620.2950,49775.0800,2020-05-07 14:36:44,2020-05-07 14:36:44,46,2.045109,0.0,0.0
18358,1115429,624,2020-05-06,685.86,6.9687,NaN,98.42,36605.9943,50726.2056,2020-05-07 14:37:12,2020-05-07 14:37:12,44,2.075521,0.0,0.0
18359,1115431,176,2020-05-06,573.94,5.7394,NaN,100.00,63081.7454,42448.6024,2020-05-07 14:37:39,2020-05-07 14:37:39,48,1.907971,0.0,0.0


In [71]:
low_loc

[854, 1055, 294, 877, 994, 996, 1189, 282, 347, 462, 828, 986, 571, 672, 903]

In [72]:
up_loc = []
low_loc = []
for i in daily_loc.pwp_loct_grp_cd.unique():
    daily_loc2 = daily_loc[(daily_loc.pwp_loct_grp_cd==i)]
    daily_loc3 = daily_loc2[daily_loc2.new_gen>0]
    upperb = np.mean(daily_loc3.new_gen)+2*np.std(daily_loc3.new_gen)
    lowerb = np.mean(daily_loc3.new_gen)-2*np.std(daily_loc3.new_gen)
    up_list = daily_loc2.pwp_id[daily_loc2.new_gen>upperb].to_list()
    low_list = daily_loc2.pwp_id[daily_loc2.new_gen<lowerb].to_list()
    up_loc += up_list
    low_loc += low_list

In [73]:
daily_pri = daily[daily.stnd_date<=base_time2]

In [76]:
# interquartile
up_pri = []
low_pri = []
common = []
tosm = []
noyes =[]
for t in daily.pwp_id.unique():
#     print(t)
    daily_pri2 = daily_pri[daily_pri.pwp_id==t]
    daily_pri2["zscore"] = (daily_pri2.avg_pv_time - np.nanmean(daily_pri2.avg_pv_time))/np.nanstd(daily_pri2.avg_pv_time)
    try:
        daily_pri_b = daily_pri2.zscore[daily_pri2.stnd_date==base_time2].values[0]
    except:
        noyes+=[t]
        next
    daily_pri_h = daily_pri2[daily_pri2.stnd_date<base_time2]
    daily_pri_h2 = daily_pri_h.zscore[daily_pri_h.new_gen>0]
    if len(daily_pri_h2)<20:
        tosm +=[t]
        next
    else:
        print(t)
        upperb = 3
        lowerb = - 3
        if daily_pri_b>upperb:
            up_pri +=[t]
        elif daily_pri_b<lowerb:
            low_pri +=[t]
        else:
            common +=[t]

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



109
208
511
284
863
242
491
683
672
63
1002
24
156
159
171
175
176
209
276
298
301
305
306
318
325
365
377
380
387
388
389
412
433
446
473
490
501
509
510
512
517
624
628
645
652
655
659
674
675
679
684
690
770
832
833
862
867
1154
1159
1075
1233
1234
48
5
8
9
10
15
18
19
20
21
23
26
28
32
33
34
35
36
37
39
40
41
43
44
45
46
47
49
50
51
52
53
61
69
70
75
77
80
82
83
87
89
95
96
98
101
102
106
107
108
110
111
115
116
121
128
132
133
136
137
143
144
149
151
154
157
160
161
165
168
172
173
174
177
178
179
186
187
188
189
191
192
193
194
210
220
223
224
225
226
241
243
247
248
249
250
251
252
253
254
257
258
268
273
277
281
283
285
287
288
293
299
300
307
308
315
316
317
321
322
326
328
333
335
339
342
343
345
347
351
352
359
364
366
367
368
369
370
371
373
374
376
379
384
386
390
393
401
402
410
415
416
418
420
428
429
430
431
434
436
437
438
439
440
447
449
453
461
462
472
474
477
479
483
486
492
503
508
513
514
516
519
520
524
529
533
536
539
540
541
542
543
547
550
552
554
555
559
560


C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning:

Mean of empty slice



In [75]:
up_pri = []
low_pri = []
common = []
tosm = []
noyes =[]
for t in daily.pwp_id.unique():
    print(t)
    daily_pri2 = daily_pri[daily_pri.pwp_id==t]
    try:
        daily_pri_b = daily_pri2.new_gen[daily_pri2.stnd_date==base_time2].values[0]
    except:
        noyes+=[t]
        next
    daily_pri_h = daily_pri2[daily_pri2.stnd_date<base_time2]
    daily_pri_h2 = daily_pri_h.new_gen[daily_pri_h.new_gen>0]
    if len(daily_pri_h2)<20:
        tosm +=[t]
        next
    else:
        print(t)
        upperb = np.mean(daily_pri_h2)+2*np.std(daily_pri_h2)
        lowerb = np.mean(daily_pri_h2)-2*np.std(daily_pri_h2)
        if daily_pri_b>upperb:
            up_pri +=[t]
        elif daily_pri_b<lowerb:
            low_pri +=[t]
        else:
            common +=[t]

218
424
556
170
109
109
208
208
511
511
669
284
284
863
863
242
242
491
491
683
683
672
672
828
63
63
1068
1122
1125
1044
1002
1002
1045
24
24
156
156
159
159
171
171
175
175
176
176
209
209
276
276
298
298
301
301
305
305
306
306
318
318
325
325
365
365
377
377
380
380
387
387
388
388
389
389
412
412
433
433
446
446
473
473
490
490
501
501
509
509
510
510
512
512
517
517
624
624
628
628
645
645
652
652
655
655
659
659
674
674
675
675
679
679
684
684
690
690
770
770
832
832
833
833
862
862
867
867
1154
1154
1159
1159
1075
1075
1233
1233
1234
1234
48
48
5
5
8
8
9
9
10
10
15
15
18
18
19
19
20
20
21
21
23
23
26
26
28
28
32
32
33
33
34
34
35
35
36
36
37
37
39
39
40
40
41
41
43
43
44
44
45
45
46
46
47
47
49
49
50
50
51
51
52
52
53
53
61
61
69
69
70
70
75
75
77
77
80
80
82
82
83
83
87
87
89
89
95
95
96
96
98
98
101
101
102
102
106
106
107
107
108
108
110
110
111
111
115
115
116
116
121
121
128
128
132
132
133
133
136
136
137
137
143
143
144
144
149
149
151
151
154
154
155
157
157
160
160
161

In [77]:
daily_loc["location"] = np.repeat(np.nan, len(daily_loc))
daily_loc["private"] = np.repeat(np.nan,len(daily_loc))

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [78]:
daily_loc.location[daily_loc.pwp_id.isin(up_loc)]=1
daily_loc.location[daily_loc.pwp_id.isin(low_loc)]=2
daily_loc.location[daily_loc.location.isnull()]=0

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [79]:
daily_loc.private[daily_loc.pwp_id.isin(up_pri)]=1
daily_loc.private[daily_loc.pwp_id.isin(low_pri)]=2
daily_loc.private[daily_loc.pwp_id.isin(noyes)]=3
daily_loc.private[daily_loc.pwp_id.isin(tosm)]=4
daily_loc.private[daily_loc.pwp_id.isin(common)]=0

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launche

In [393]:
daily_loc[daily_loc.pwp_id==5]

,id,pwp_id,stnd_date,pv_amt,avg_pv_time,rnk,tot_pv_cpct,rec_amt,smp_amt,rpt_dtm,reg_dtm,pwp_loct_grp_cd,new_gen,location,private
18413,1102102,5,2020-05-06,0.0,0.0,NaN,100.0,0.0,0.0,2020-05-06 17:30:00,2020-05-06 18:00:01,45,0.0,2.0,2.0


In [84]:
daily_loc[(daily_loc.location==2)&(daily_loc.private==0)]

,id,pwp_id,stnd_date,pv_amt,avg_pv_time,rnk,tot_pv_cpct,rec_amt,smp_amt,rpt_dtm,reg_dtm,pwp_loct_grp_cd,new_gen,location,private
17773,1106626,21,2020-05-06,365.00,3.6558,NaN,99.84,19480.9260,26995.4000,2020-05-06 21:30:00,2020-05-06 22:00:01,43,1.538114,2.0,0.0
17887,1106740,315,2020-05-06,777.00,2.5900,NaN,300.00,36839.4985,57466.9200,2020-05-06 21:30:00,2020-05-06 22:00:01,50,1.278152,2.0,0.0
17900,1106753,347,2020-05-06,0.00,0.0000,NaN,93.00,0.0000,0.0000,2020-05-06 21:30:00,2020-05-06 22:00:01,42,0.000000,2.0,0.0
17920,1106773,402,2020-05-06,508.00,5.0800,NaN,100.00,78843.6320,37571.6800,2020-05-06 21:30:00,2020-05-06 22:00:01,45,1.805005,2.0,0.0
17933,1106786,438,2020-05-06,666.00,3.9880,NaN,167.00,33146.6622,49257.3600,2020-05-06 21:30:00,2020-05-06 22:00:01,47,1.607035,2.0,0.0
17940,1106793,462,2020-05-06,0.00,0.0000,NaN,395.91,0.0000,0.0000,2020-05-06 21:30:00,2020-05-06 22:00:01,42,0.000000,2.0,0.0
17976,1106829,569,2020-05-06,2574.40,5.5254,NaN,465.92,171752.3832,190402.6240,2020-05-06 21:30:00,2020-05-06 22:00:01,46,1.875702,2.0,0.0
17977,1106830,570,2020-05-06,2398.70,5.3238,NaN,450.56,160030.4699,177407.8520,2020-05-06 21:30:00,2020-05-06 22:00:01,46,1.844320,2.0,0.0
17978,1106831,571,2020-05-06,2552.80,5.1401,NaN,496.64,170311.3284,188805.0880,2020-05-06 21:30:00,2020-05-06 22:00:01,46,1.814841,2.0,0.0
17989,1106842,598,2020-05-06,489.00,4.8900,NaN,100.00,26077.3543,36166.4400,2020-05-06 21:30:00,2020-05-06 22:00:01,45,1.773256,2.0,0.0


In [356]:
daily_loc[(daily_loc.location==0) &(daily_loc.private==1)]

,id,pwp_id,stnd_date,pv_amt,avg_pv_time,rnk,tot_pv_cpct,rec_amt,smp_amt,rpt_dtm,reg_dtm,pwp_loct_grp_cd,new_gen,location,private


In [63]:
low_loc

[700,
 924,
 1189,
 1278,
 1279,
 1280,
 1281,
 803,
 854,
 1055,
 294,
 877,
 994,
 996,
 347,
 462,
 828,
 986,
 21,
 282,
 903]

In [20]:
up_pri

[]

In [256]:
low_pri

[462, 877, 1055, 5]

In [271]:
sorted(common)

[8,
 9,
 10,
 15,
 18,
 19,
 20,
 21,
 23,
 24,
 26,
 28,
 32,
 33,
 34,
 35,
 36,
 37,
 39,
 40,
 41,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 61,
 69,
 70,
 75,
 77,
 80,
 82,
 83,
 87,
 89,
 95,
 96,
 98,
 101,
 102,
 106,
 107,
 108,
 109,
 110,
 111,
 115,
 116,
 121,
 128,
 132,
 133,
 136,
 137,
 143,
 144,
 149,
 151,
 154,
 156,
 157,
 159,
 160,
 161,
 165,
 168,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 186,
 187,
 188,
 189,
 191,
 192,
 193,
 194,
 208,
 209,
 210,
 220,
 223,
 224,
 225,
 226,
 241,
 242,
 243,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 257,
 258,
 268,
 273,
 276,
 277,
 283,
 284,
 285,
 287,
 288,
 293,
 298,
 299,
 300,
 301,
 305,
 306,
 307,
 308,
 315,
 316,
 317,
 318,
 321,
 322,
 325,
 326,
 328,
 333,
 335,
 339,
 342,
 343,
 345,
 347,
 351,
 352,
 359,
 364,
 366,
 367,
 368,
 369,
 370,
 371,
 373,
 374,
 376,
 377,
 379,
 380,
 384,
 386,
 387,
 388,
 389,
 390,
 393,
 401,
 402,
 410,
 412,
 415,
 416,
 41

In [259]:
noyes

[124,
 126,
 222,
 719,
 864,
 411,
 522,
 218,
 170,
 669,
 556,
 562,
 357,
 424,
 105,
 1012,
 1044,
 1068,
 1122,
 1125,
 1045,
 298,
 1198,
 1059]

In [272]:
sorted(tosm)

[63,
 105,
 124,
 126,
 155,
 170,
 218,
 222,
 281,
 282,
 294,
 357,
 411,
 424,
 522,
 556,
 562,
 657,
 669,
 719,
 828,
 864,
 986,
 996,
 1012,
 1044,
 1045,
 1059,
 1068,
 1122,
 1125,
 1189,
 1198,
 1216,
 1217,
 1227,
 1229,
 1263]

In [207]:
up_loc

[616, 365, 786]

In [208]:
low_loc

[155,
 657,
 438,
 903,
 663,
 828,
 880,
 901,
 945,
 985,
 986,
 992,
 5,
 402,
 598,
 803,
 1055,
 462,
 662,
 294,
 877,
 994,
 996,
 1189,
 21,
 282,
 614]

In [94]:
accu = pd.read_sql("SELECT * FROM generation_log_hourly_accumulate where stnd_dtm < '2020-04-28 00:00:00' and stnd_dtm >='2020-04-21 00:00:00'",con=con_saem)

In [37]:
accu

,id,pwp_id,stnd_dtm,pv_amt,reg_dtm
0,150979,21,2020-04-27 00:30:00,0.0,2020-04-27 00:30:25
1,150980,121,2020-04-27 00:30:00,0.0,2020-04-27 00:30:25
2,150981,300,2020-04-27 00:30:00,0.0,2020-04-27 00:30:27
3,150982,651,2020-04-27 00:30:00,0.0,2020-04-27 00:30:27
4,150983,808,2020-04-27 00:30:00,0.0,2020-04-27 00:30:27
...,...,...,...,...,...
12073,163052,836,2020-04-27 23:30:00,649.0,2020-04-27 23:33:22
12074,163053,875,2020-04-27 23:30:00,419.2,2020-04-27 23:33:22
12075,163054,926,2020-04-27 23:30:00,390.4,2020-04-27 23:33:25
12076,163055,102,2020-04-27 23:30:00,500.2,2020-04-27 23:33:25


In [44]:
accu[accu.pwp_id==700][96:144]

,id,pwp_id,stnd_dtm,pv_amt,reg_dtm
49448,129826,700,2020-04-25 05:30:00,0.0,2020-04-25 05:31:10
49922,130300,700,2020-04-25 06:30:00,0.0,2020-04-25 06:31:03
50418,130796,700,2020-04-25 07:30:00,0.0,2020-04-25 07:31:10
50892,131270,700,2020-04-25 08:30:00,30.6,2020-04-25 08:31:03
51429,131807,700,2020-04-25 09:30:00,68.3,2020-04-25 09:31:10
51925,132303,700,2020-04-25 10:30:00,124.0,2020-04-25 10:31:07
52376,132754,700,2020-04-25 11:30:00,201.6,2020-04-25 11:31:07
52886,133264,700,2020-04-25 12:30:00,284.2,2020-04-25 12:31:03
53399,133777,700,2020-04-25 13:30:00,365.5,2020-04-25 13:31:07
53895,134273,700,2020-04-25 14:30:00,441.7,2020-04-25 14:31:01


In [153]:
gdaily = pd.merge(daily,pm,on=["pwp_id"])

In [160]:
jdaily = gdaily[gdaily.pwp_loct_grp_cd=="41"].reset_index(drop=True)

In [161]:
jdaily.avg_pv_time

0     3.8312
1     4.7714
2     5.8969
3     4.1146
4     6.2600
       ...  
69    6.7015
70    6.7197
71    4.8995
72    5.9118
73    4.7510
Name: avg_pv_time, Length: 74, dtype: float64

In [87]:
np.where(jdaily.avg_pv_time!=np.max(jdaily.avg_pv_time))[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68,
       69], dtype=int64)

In [162]:
jdaily["new_gen"] = np.log(jdaily.avg_pv_time+1)
bbbbbb = jdaily[jdaily.new_gen>0]

In [99]:
jdaily.avg_pv_time[np.where(np.max(jdaily.avg_pv_time))]

ValueError: Can only tuple-index with a MultiIndex

In [163]:
np.std(jdaily.new_gen)

0.39675727374891134

In [164]:
lb = np.mean(bbbbbb.new_gen)-2*np.std(bbbbbb.new_gen)
ub =  np.mean(bbbbbb.new_gen)+2*np.std(bbbbbb.new_gen)

In [165]:
jdaily[jdaily.new_gen<lb]

,id,pwp_id,stnd_date,pv_amt,avg_pv_time,rnk,tot_pv_cpct_x,rec_amt,smp_amt,rpt_dtm,...,oprt_strt_date,reg_prct_step,inverters,use_yn,del_yn,mdf_id,mdf_dtm,reg_id,reg_dtm_y,new_gen
16,963561,294,2020-04-27,0.0,0.0000,None,34.32,0.0000,0.00,2020-04-28 18:12:29,...,2017-12-21,2,None,Y,N,-1,2020-04-02 17:42:27,0,2020-04-01 00:54:01,0.000000
52,972655,987,2020-04-27,83.0,1.6768,None,49.50,4431.1044,6172.71,2020-04-28 18:43:11,...,None,2,None,Y,N,-1,2020-04-02 19:07:23,0,2020-04-02 00:17:18,0.984622
53,972683,988,2020-04-27,79.0,1.5960,None,49.50,4217.5572,5875.23,2020-04-28 18:43:13,...,None,2,None,Y,N,-1,2020-04-02 19:07:23,0,2020-04-02 00:17:18,0.953972
55,972739,990,2020-04-27,72.0,1.4545,None,49.50,3843.8496,5354.64,2020-04-28 18:43:18,...,None,2,None,Y,N,-1,2020-04-02 19:07:23,0,2020-04-02 00:17:19,0.897923
56,972851,994,2020-04-27,16.0,0.4618,None,34.65,854.1888,1189.92,2020-04-28 18:43:28,...,None,2,None,Y,N,-1,2020-04-02 19:07:23,0,2020-04-02 00:17:21,0.379669
58,972907,996,2020-04-27,0.0,0.0000,None,49.50,0.0000,0.00,2020-04-28 18:43:33,...,None,2,None,Y,N,-1,2020-04-02 19:07:23,0,2020-04-02 00:17:21,0.000000
